# Notebook Details
This notebook enhances training labels by applying pseudo-labeling to the False Positive (FP) regions identified from the original Mask2Former model, using the Segment Anything Model (SAM) for refinement.

At the end of this notebook, a new folder named pseudo_dataset will be generated, containing four folders: (i) train_image (ii) train_mask (iii) val_image (iv) val_mask

* train_image: Same images as Input_data/train_image
* val_image: GT + pseudo-labeled masks (FP corrected)
* train_mask: Same images as Input_data/val_image
* val_mask: GT + pseudo-labeled masks (FP corrected)




**IMPORTANT: This notebook is solely for generating pseudo and ground truth (GT) masks. To retrain the model using the enhanced masks, please use "Notebook 2: Mask2Former" and update the training input directory from "Input_data" to "pseudo_dataset".**

**Note:**
1. "<<<" in the code indicates values that you can modify.

2. This notebook currently focuses on False Positives only, but can be modified to target False Negatives.


    




# Environment Setup & Imports

In [ ]:
# Install dependencies if not yet installed
# Only run this section if needed.

# Set UTF-8 locale to avoid encoding-related errors (e.g., with gsutil)
import os
os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"

# Install basic dependencies: OpenCV for image processing, matplotlib for visualization
!pip install opencv-python matplotlib

# Clone the Segment Anything repository from GitHub (Meta AI)
!git clone https://github.com/facebookresearch/segment-anything.git
%cd segment-anything
!pip install -e .

# Download the pre-trained SAM (Segment Anything Model) ViT-B weights
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O /content/segment-anything/sam_vit_b_01ec64.pth


Cloning into 'segment-anything'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 304 (delta 2), reused 1 (delta 1), pack-reused 299 (from 2)
Receiving objects: 100% (304/304), 18.31 MiB | 34.28 MiB/s, done.
Resolving deltas: 100% (159/159), done.
/content/segment-anything
Obtaining file:///content/segment-anything
  Preparing metadata (setup.py) ... done
  Running setup.py develop for segment_anything
--2025-05-21 10:17:31--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.44, 18.238.176.126, 18.238.176.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘/content/segment-anything/sam_vit_b_01ec64.pth’

/content/segment-an 100%[================

In [ ]:
#Change to access your bucket or path
!gsutil -m cp -r gs://yen_bucket/directory_structure/content/* /content/

# Utility Functions & Model Setup


This section sets up important parameters such as image size, confidence thresholds, and Input/Output paths. It also defines two key functions:



*   generate_pseudo_negative_mask() for SAM segmentation.
*   generate_final_mask() to merge GT and pseudo labels.





In [ ]:
# install all neccessary libraries, delete when neccessary
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 32.4 MB/s eta 0:00:00


In [ ]:
# === [Package Locking: ensure stable versions]
!pip install -U "dill==0.3.6" "evaluate==0.4.0" --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 22.2 MB/s eta 0:00:00


In [ ]:

import os
import cv2
import numpy as np
import torch
from tqdm import tqdm

from segment_anything import SamPredictor, sam_model_registry
from transformers import Mask2FormerForUniversalSegmentation, Mask2FormerImageProcessor
import evaluate
import matplotlib.pyplot as plt

# === Select the appropriate device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# === Load the SAM model using 'vit_b' (base variant) and the pre-trained checkpoint
sam = sam_model_registry["vit_b"](checkpoint="/content/segment-anything/sam_vit_b_01ec64.pth").to(device)
# === Create a predictor instance to run SAM inference later
predictor = SamPredictor(sam)

/content/segment-anything/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [ ]:
def generate_pseudo_negative_mask(image, fp_mask, predictor, score_threshold=0.8, area_threshold=100):
    """
    Generate a pseudo negative mask using SAM clipped to FP area.

    Args:
        image (np.ndarray): RGB image.
        fp_mask (np.ndarray): Binary mask where FP area is 255.
        predictor (SamPredictor): SAM predictor instance.
        score_threshold (float): SAM mask confidence score threshold.
        area_threshold (int): Minimum size of connected components to keep.

    Returns:
        np.ndarray: Pseudo-negative mask (uint8, 0 or 255).
    """
    # Erode FP mask to reduce border noise
    kernel = np.ones((3, 3), np.uint8)
    fp_mask_eroded = cv2.erode(fp_mask, kernel, iterations=1)

    predictor.set_image(image)
    ys, xs = np.where(fp_mask_eroded == 255)

    if len(xs) == 0:
        return np.zeros_like(fp_mask)

    # Bounding box as prompt
    x1, y1, x2, y2 = np.min(xs), np.min(ys), np.max(xs), np.max(ys)
    input_box = np.array([x1, y1, x2, y2])

    masks, scores, _ = predictor.predict(
        box=input_box[None, :],
        multimask_output=True
    )

    best_mask = masks[np.argmax(scores)]
    best_score = np.max(scores)

    if best_score < score_threshold:
        return np.zeros_like(fp_mask)

    # Only keep confident SAM mask clipped by FP
    pseudo_negative_mask = np.logical_and(best_mask, fp_mask > 0).astype(np.uint8) * 255

    # Remove small noise areas
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(pseudo_negative_mask, connectivity=8)
    cleaned_mask = np.zeros_like(pseudo_negative_mask)
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area > area_threshold:
            cleaned_mask[labels == i] = 255

    return cleaned_mask


In [ ]:
def generate_final_mask(gt_mask, pred_mask, pseudo_negative_mask, save_path=None):
    """
    Combine GT and pseudo-negative mask into final mask and optionally save it.

    Args:
        gt_mask (np.ndarray): Ground truth binary mask (0/1).
        pred_mask (np.ndarray): Predicted mask from model (0/1).
        pseudo_negative_mask (np.ndarray): Pseudo mask from SAM (0/255).
        save_path (str, optional): Path to save the final mask. If None, mask is not saved.

    Returns:
        np.ndarray: Final mask for training (values: 0, 1, 255).
    """
    # Shape alignment
    if pseudo_negative_mask.shape != gt_mask.shape:
        pseudo_negative_mask = cv2.resize(
            pseudo_negative_mask,
            (gt_mask.shape[1], gt_mask.shape[0]),
            interpolation=cv2.INTER_NEAREST
        )

    final_mask = np.full_like(gt_mask, fill_value=255)

    # Keep true buildings
    final_mask[gt_mask == 1] = 1

    # Replace confident FP regions with background (0)
    fp_area = ((pred_mask == 1) & (gt_mask == 0))
    sam_fp_area = ((pseudo_negative_mask == 255) & fp_area)
    final_mask[sam_fp_area] = 0

    # Optional save
    if save_path is not None:
      os.makedirs(os.path.dirname(save_path), exist_ok=True)
      cv2.imwrite(save_path, final_mask)

# Pseudo-Label Generation Loop

This section performs the core logic:

Loads images and GT masks from Input_data

*   Runs Mask2Former to generate predictions
*   Computes mIoU and identifies poor predictions
*   Applies SAM on FP regions when necessary
*   Saves results into pseudo_dataset

Both train and val sets are processed.

In [ ]:
# Return to the root content directory (to avoid path issues)
%cd /content

/content


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# === Default parameters
miou_threshold = 0.7           #<<< Skip SAM if mIoU is already good
score_threshold = 0.8          #<<< Confidence threshold for SAM masks
area_threshold = 100           #<<< Minimum area (in pixels) for valid SAM regions
image_size = (256, 256)        #<<< Resize target size for all inputs

# Load trained Mask2Former model (modify path if needed)
inf_model = Mask2FormerForUniversalSegmentation.from_pretrained("outputs/model_iou").to(device).eval()

# Image processor to format input image
processor = Mask2FormerImageProcessor()

# Evaluation metric (e.g., mIoU)
infer_metric = evaluate.load("mean_iou")

# === Process both train and val sets
for split in ["train", "val"]:
    print(f"\nProcessing {split} set...")

    input_img_dir = os.path.join("Input_data", f"{split}_image")
    input_mask_dir = os.path.join("Input_data", f"{split}_mask")

    output_img_dir = os.path.join("pseudo_dataset", f"{split}_image")
    output_mask_dir = os.path.join("pseudo_dataset", f"{split}_mask")

    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)

    image_files = sorted(os.listdir(input_img_dir))

    for img_file in tqdm(image_files):
        # === 1. Load image and GT mask
        img_path = os.path.join(input_img_dir, img_file)
        mask_path = os.path.join(input_mask_dir, img_file.replace("img", "mask"))

        orig_image = cv2.imread(img_path)
        orig_image = cv2.resize(orig_image, image_size)
        image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)

        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = cv2.resize(gt_mask, image_size)

        # === 2. Predict with Mask2Former
        inputs = processor(images=image, return_tensors="pt").to(device)
        outputs = inf_model(**inputs)
        labels = processor.post_process_semantic_segmentation(outputs, target_sizes=[image_size])[0]
        pred_mask = labels.cpu().numpy()

        # === 3. Compute mIoU
        infer_metric.add_batch(references=[gt_mask], predictions=[pred_mask])
        miou = infer_metric.compute(num_labels=2, ignore_index=255, reduce_labels=False)['mean_iou']

        # === 4. Save original mask if mIoU >= threshold
        out_img_path = os.path.join(output_img_dir, img_file)
        out_mask_path = os.path.join(output_mask_dir, img_file.replace("img", "mask"))

        if miou >= miou_threshold:
            cv2.imwrite(out_img_path, orig_image)
            cv2.imwrite(out_mask_path, gt_mask)
            continue

        # === 5. Generate pseudo-negative mask using SAM
        # Default: False Positive (predicted as 1, but GT is 0)
        fp_mask = ((pred_mask == 1) & (gt_mask == 0)).astype(np.uint8) * 255
        # Optional: Target False Negatives instead (predicted as 0, but GT is 1)
        # fn_mask = ((pred_mask == 0) & (gt_mask == 1)).astype(np.uint8) * 255

        pseudo_negative_mask = generate_pseudo_negative_mask(
            image=image,
            fp_mask=fp_mask,
            predictor=predictor,
            score_threshold=score_threshold,
            area_threshold=area_threshold
        )

        # === 6. Merge GT and pseudo mask, then save
        cv2.imwrite(out_img_path, orig_image)

        final_mask = generate_final_mask(
            gt_mask=gt_mask,
            pred_mask=pred_mask,
            pseudo_negative_mask=pseudo_negative_mask,
            save_path=out_mask_path
        )



Processing train set...


  4%|▎         | 22/600 [00:11<04:08,  2.33it/s]/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
  9%|▊         | 52/600 [00:23<03:01,  3.01it/s]/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
100%|██████████| 600/600 [03:35<00:00,  2.78it/s]



Processing val set...


100%|██████████| 200/200 [01:11<00:00,  2.80it/s]
